In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configure display settings
pd.set_option('display.max_columns', None)
plt.style.use('ggplot')

In [ ]:
# Load historical trader data
historical_df = pd.read_csv("/content/drive/MyDrive/ds_Rahul/csv_files/historical_data.csv")

# Load Fear & Greed index data
fear_greed_df = pd.read_csv("/content/drive/MyDrive/ds_Rahul/csv_files/fear_greed_index.csv")

# Check basic info
print("Historical Data Shape:", historical_df.shape)
print("Fear & Greed Data Shape:", fear_greed_df.shape)

In [ ]:
# View first few rows
display(historical_df.head())

In [ ]:
# Summary statistics
display(historical_df.describe())

In [ ]:
# View first few rows
display(fear_greed_df.head())

In [ ]:
# Summary statistics
display(fear_greed_df.describe())

In [ ]:
#Data Cleaning
# --- Historical Trader Data ---
# Convert timestamp to datetime
historical_df['Timestamp IST'] = pd.to_datetime(historical_df['Timestamp IST'], format='%d-%m-%Y %H:%M', errors='coerce')

# Convert Unix timestamp to datetime (if present)
if historical_df['Timestamp'].notnull().sum() > 0:
    historical_df['Timestamp'] = pd.to_datetime(historical_df['Timestamp'], unit='s', errors='coerce')

# Clean categorical fields
historical_df['Side'] = historical_df['Side'].str.upper()

# Fill missing numeric values
historical_df.fillna({
    'Closed PnL': 0.0,
    'Fee': 0.0,
    'Size Tokens': 0.0,
    'Size USD': 0.0
}, inplace=True)

# Create a 'date' column for merging
historical_df['date'] = historical_df['Timestamp IST'].dt.date
historical_df['date'] = pd.to_datetime(historical_df['date'])

In [ ]:
#Data Cleaning
# --- Fear & Greed Data ---
fear_greed_df['date'] = pd.to_datetime(fear_greed_df['date'], format='%Y-%m-%d', errors='coerce')
fear_greed_df['classification'] = fear_greed_df['classification'].str.title()

In [ ]:
#Merge the Datasets
merged_df = pd.merge(
    historical_df,
    fear_greed_df[['date', 'value', 'classification']],
    on='date',
    how='left'
)

print("Merged Data Shape:", merged_df.shape)
display(merged_df.head())

In [ ]:
#Save the Cleaned Dataset
merged_df.to_csv(f"/content/drive/MyDrive/ds_Rahul/csv_files/cleaned_merged_data.csv", index=False)
print("Cleaned merged data saved in csv_files folder.")

In [ ]:
#Basic EDA
# A. Buy vs Sell Count

buy_sell_counts = merged_df['Side'].value_counts()

plt.figure(figsize=(6,4))
sns.barplot(x=buy_sell_counts.index, y=buy_sell_counts.values)
plt.title("Buy vs Sell Trade Count")
plt.xlabel("Trade Side")
plt.ylabel("Count")
plt.savefig(f"/content/drive/MyDrive/ds_Rahul/outputs/buy_vs_sell.png")
plt.show()

In [ ]:
# B. Daily Trading Volume

daily_volume = merged_df.groupby('date')['Size USD'].sum()

plt.figure(figsize=(10,5))
daily_volume.plot()
plt.title("Daily Total Trading Volume (USD)")
plt.xlabel("Date")
plt.ylabel("Total Volume (USD)")
plt.savefig(f"/content/drive/MyDrive/ds_Rahul/outputs/daily_volume.png")
plt.show()

In [ ]:
#Profitability vs Sentiment Analysis

#Analyze profitability by sentiment type (Fear, Greed, etc.).

# Group by sentiment classification
profitability_stats = merged_df.groupby('classification').agg(
    total_trades=('Account', 'count'),
    total_pnl=('Closed PnL', 'sum'),
    avg_pnl=('Closed PnL', 'mean'),
    total_volume_usd=('Size USD', 'sum'),
    avg_trade_volume_usd=('Size USD', 'mean')
).reset_index()

display(profitability_stats)

In [ ]:
# A. Total PnL by sentiment
pnl_by_sentiment = merged_df.groupby('classification')['Closed PnL'].sum().reset_index()

# Sort for better visualization (highest to lowest total PnL)
pnl_by_sentiment = pnl_by_sentiment.sort_values(by='Closed PnL', ascending=False)

# Plot
plt.figure(figsize=(7,4))
sns.barplot(x='classification', y='Closed PnL', data=pnl_by_sentiment, palette='viridis')

plt.title("Total Closed PnL by Market Sentiment")
plt.xlabel("Market Sentiment")
plt.ylabel("Total Closed PnL (USD)")
plt.xticks(rotation=15)
plt.tight_layout()

# Save to outputs folder
plt.savefig("/content/drive/MyDrive/ds_Rahul/outputs/pnl_by_sentiment.png")
plt.show()

print("pnl_by_sentiment.png saved in outputs folder.")

In [ ]:
# B. Average PnL by Sentiment

plt.figure(figsize=(7,4))
sns.barplot(x='classification', y='avg_pnl', data=profitability_stats, palette='coolwarm')
plt.title("Average Closed PnL per Trade by Market Sentiment")
plt.xlabel("Market Sentiment")
plt.ylabel("Average Closed PnL")
plt.savefig(f"/content/drive/MyDrive/ds_Rahul/outputs/avg_pnl_by_sentiment.png")
plt.show()

In [ ]:
# C. Total Trade Volume by Sentiment

plt.figure(figsize=(7,4))
sns.barplot(x='classification', y='total_volume_usd', data=profitability_stats, palette='muted')
plt.title("Total Trade Volume by Market Sentiment (USD)")
plt.xlabel("Market Sentiment")
plt.ylabel("Total Volume (USD)")
plt.savefig(f"/content/drive/MyDrive/ds_Rahul/outputs/volume_by_sentiment.png")
plt.show()

In [ ]:
# Export Final Report Data

summary = {
    "total_trades": int(merged_df.shape[0]),
    "total_volume_usd": float(merged_df['Size USD'].sum()),
    "total_closed_pnl": float(merged_df['Closed PnL'].sum()),
    "buy_vs_sell": buy_sell_counts.to_dict(),
    "profitability_by_sentiment": profitability_stats.to_dict(orient='records')
}

# Save summary as CSV for reporting
pd.DataFrame.from_dict(summary['profitability_by_sentiment']).to_csv(
    f"/content/drive/MyDrive/ds_Rahul/csv_files/profitability_summary.csv", index=False
)

print("Summary data exported successfully!")